In [ ]:
def get_sparse_tensor(e, KG):
    print('getting a sparse tensor...')
    M0, du = get_vmat(e, KG)

    # New Addition here    # Abductive Reasoning
    G = nx.Graph()
    G.add_edges_from(M0)
    #nx.number_of_nodes(G), nx.number_of_edges(G)

    #split list of highest cent nodes into all possible subsets size 2
    HCNchoose2 = list(itertools.combinations(getHighestCentralityNodes(G), 2))

    #Similiarity Calculation
    preds = nx.jaccard_coefficient(G, HCNchoose2)

    M2 = {}
    for u, v, p in preds:
        if (u, v) not in M2 and float(f" {p:.8f}")>0:
            M2[(u, v)] = float(f" {p:.8f}")    #Predicted relationship semantics. "M2" are added to "M0"
    M0 = {**M0, **M2}                 
    # New AdditionEnd here

    ind = []
    val = []
    
    M0Pind = []
    M0Pval = []
    
    M0_list = list(M0.keys()) # convert "M0" "dict" to "list"
    s_column = [item[1] for item in M0_list]

    for fir, sec in M0: #Check each relation triple / complex relationship explanation in dict
#Partial Explanations
        if fir in s_column:
            M0Pind.append((sec, fir))
            M0Pval.append(M0[(fir, sec)] / math.sqrt(du[fir]) / math.sqrt(du[sec]))   #value od "d" 1/xyz
            
#Complete Explanations
        if fir in s_column or fir not in s_column:
            ind.append((sec, fir))        #Indices / Location
            val.append(M0[(fir, sec)] / math.sqrt(du[fir]) / math.sqrt(du[sec]))   #value od "d" 1/xyz

    M = tf.SparseTensor(indices=ind, values=val, dense_shape=[e, e])

    return M0, M